In [2]:
import pandas as pd


matching_file = r"..\result\매칭정보.csv"

# CSV 파일 읽기 (헤더 없이)
MATCH_TABLE = pd.read_csv(matching_file)
MATCH_TABLE

,공공건축물_PK,관리_건축물_유형,건축물_구분,건축물_종류,MGM_BLD_PK,RECAP_PK,에너지공급기관코드,에너지고유번호
0,972,1,1,1,11110-900,11110-900,1100100000,149870271
1,972,1,1,1,11110-900,11110-900,1100100000,158469849
2,972,1,1,1,11110-900,11110-900,1100100000,159134691
3,972,1,1,1,11110-900,11110-900,1100100000,108601905
4,972,1,1,1,11110-900,11110-900,1100100000,141766293
...,...,...,...,...,...,...,...,...
270876,10000006164726,1,1,2,36110-1000000000000000571160,36110-1000000000000000571160,1100100000,646717914
270877,10000006164726,1,1,2,36110-1000000000000000571160,36110-1000000000000000571160,1100100000,646222299
270878,10000006164726,1,1,2,36110-1000000000000000571160,36110-1000000000000000571160,1100100000,645847562
270879,10000006164726,1,1,2,36110-1000000000000000571160,36110-1000000000000000571160,1100100000,646265242


In [3]:
# =============================================================================
# STEP 1. A1 테이블 생성
# 그룹 기준: [MGM_BLD_PK, RECAP_PK, 건축물_구분, 건축물_종류, 에너지공급기관코드]
# 각 그룹의 행 개수를 계산하여 'N' 컬럼에 저장하고,
# 두 개의 에너지 관련 플래그 컬럼을 추가합니다.
# - ENGY_YN : 모든 행에 대해 에너지가 연결되어 있다고 가정(값 1)
# - ENGY_MGM_YN : 건축물_종류가 1(총괄표제부)인 경우 0, 그 외(표제부 또는 전유부)는 1
# =============================================================================

A1 = (MATCH_TABLE
      .groupby(['MGM_BLD_PK', 'RECAP_PK', '건축물_구분', '건축물_종류', '에너지공급기관코드'], as_index=False)
      .size()
      .rename(columns={'size': 'N'}))

# 모든 행에 대해 에너지 연결 플래그를 1로 지정
A1['ENGY_YN'] = 1

# 건축물_종류가 1이면(총괄표제부) 0, 그렇지 않으면(표제부/전유부) 1로 설정
A1['ENGY_MGM_YN'] = A1['건축물_종류'].apply(lambda x: 1 if x != 1 else 0)
A1

,MGM_BLD_PK,RECAP_PK,건축물_구분,건축물_종류,에너지공급기관코드,N,ENGY_YN,ENGY_MGM_YN
0,11110-100180009,11110-100180009,2,3,1100100000,5,1,1
1,11110-100180009,11110-100180009,2,3,1200800000,1,1,1
2,11110-100180042,11110-100180042,1,1,1100100000,13,1,0
3,11110-100180042,11110-100180042,1,1,1200800000,10,1,0
4,11110-100180098,11110-100180098,1,1,1100100000,5,1,0
...,...,...,...,...,...,...,...,...
13808,52800-100223481,52800-100223481,1,1,1100100000,1,1,0
13809,52800-18617,52800-18617,1,2,1100100000,4,1,1
13810,52800-209,52800-209,1,1,1100100000,2,1,0
13811,52800-4852,52800-4852,1,2,1100100000,4,1,1


In [4]:

# =============================================================================
# STEP 2. A2 테이블 생성
# 그룹 기준: [RECAP_PK, 건축물_구분, 건축물_종류, 에너지공급기관코드]
# 각 그룹의 행 개수를 계산 후 에너지 플래그를 추가하고,
# 총괄표제부(건축물_종류 == 1)인 행만 남깁니다.
# 최종적으로 RECAP_PK, 에너지공급기관코드, ENGY_RECAP_YN 컬럼만 선택합니다.
# =============================================================================

A2 = (MATCH_TABLE
      .groupby(['RECAP_PK', '건축물_구분', '건축물_종류', '에너지공급기관코드'], as_index=False)
      .size()
      .rename(columns={'size': 'N'}))

# 모든 그룹에 대해 에너지 연결 플래그를 1로 지정
A2['ENGY_YN'] = 1

# 총괄표제부인 경우 (건축물_종류 == 1) ENGY_RECAP_YN 값을 1로, 아니면 0으로 설정
A2['ENGY_RECAP_YN'] = A2['건축물_종류'].apply(lambda x: 1 if x == 1 else 0)

# 총괄표제부(건축물_종류 == 1)인 행만 선택
A2 = A2[A2['건축물_종류'] == 1].copy()

# 최종적으로 RECAP_PK, 에너지공급기관코드, ENGY_RECAP_YN 컬럼만 선택
A2 = A2[['RECAP_PK', '에너지공급기관코드', 'ENGY_RECAP_YN']]
A2

,RECAP_PK,에너지공급기관코드,ENGY_RECAP_YN
2,11110-100180042,1100100000,1
3,11110-100180042,1200800000,1
4,11110-100180098,1100100000,1
7,11110-100183537,1100100000,1
8,11110-100183537,1200800000,1
...,...,...,...
5309,52790-687,1203700000,1
5312,52790-80,1100100000,1
5313,52790-80,1203700000,1
5317,52800-100223481,1100100000,1


In [6]:
A2['ENGY_RECAP_YN'].value_counts()

ENGY_RECAP_YN
1    1824
Name: count, dtype: int64

In [7]:

# =============================================================================
# STEP 3. A3 테이블 생성
# A1에서 표제부에 해당하는 행을 추출합니다.
# (표제부: 건축물_종류가 2 또는 3인 경우)
# 이후 A2의 총괄표제부 정보를 'RECAP_PK'와 '에너지공급기관코드'를 기준으로 병합합니다.
# =============================================================================

# 건축물_종류가 2 또는 3인 행만 선택 → 표제부(단일 건물 또는 집합 건물의 표제부)
A3 = A1[(A1['건축물_종류'] == 2) | (A1['건축물_종류'] == 3)].copy()

# A2와 'RECAP_PK', '에너지공급기관코드' 기준으로 왼쪽 병합
A3 = pd.merge(A3, A2, on=['RECAP_PK', '에너지공급기관코드'], how='left')
A3

,MGM_BLD_PK,RECAP_PK,건축물_구분,건축물_종류,에너지공급기관코드,N,ENGY_YN,ENGY_MGM_YN,ENGY_RECAP_YN
0,11110-100180009,11110-100180009,2,3,1100100000,5,1,1,NaN
1,11110-100180009,11110-100180009,2,3,1200800000,1,1,1,NaN
2,11110-100192417,11110-100192417,1,2,1100100000,8,1,1,NaN
3,11110-100196007,11110-100196007,1,2,1100100000,7,1,1,NaN
4,11110-100196007,11110-100196007,1,2,1200800000,3,1,1,NaN
...,...,...,...,...,...,...,...,...,...
4491,52800-100182279,52800-100182279,1,2,1203600000,3,1,1,NaN
4492,52800-100182788,52800-100182788,1,2,1100100000,3,1,1,NaN
4493,52800-18617,52800-18617,1,2,1100100000,4,1,1,NaN
4494,52800-4852,52800-4852,1,2,1100100000,4,1,1,NaN


In [8]:

# =============================================================================
# STEP 4. A4 테이블 생성
# 그룹 기준: [RECAP_PK, 건축물_구분, 건축물_종류, 에너지공급기관코드]
# 총괄표제부(건축물_종류 == 1)인 행만 선택한 후,
# 단지(RECAP_PK)별 중복을 제거하고, RECAP_YN 플래그(값 1)를 추가합니다.
# 최종적으로 RECAP_PK와 RECAP_YN 컬럼만 남깁니다.
# =============================================================================

A4 = (MATCH_TABLE
      .groupby(['RECAP_PK', '건축물_구분', '건축물_종류', '에너지공급기관코드'], as_index=False)
      .size()
      .rename(columns={'size': 'N'}))

# 총괄표제부인 행만 선택 (건축물_종류 == 1)
A4 = A4[A4['건축물_종류'] == 1].copy()

# RECAP_PK 기준 중복 제거 (한 단지 당 한 행만 남김)
A4 = A4.drop_duplicates(subset=['RECAP_PK'])

# 단지에 총괄표제부가 존재함을 나타내는 RECAP_YN 플래그를 1로 지정
A4['RECAP_YN'] = 1

# 최종적으로 RECAP_PK와 RECAP_YN 컬럼만 선택
A4 = A4[['RECAP_PK', 'RECAP_YN']]
A4

,RECAP_PK,RECAP_YN
2,11110-100180042,1
4,11110-100180098,1
7,11110-100183537,1
9,11110-100187671,1
12,11110-100190988,1
...,...,...
5307,52790-675,1
5308,52790-687,1
5312,52790-80,1
5317,52800-100223481,1


In [9]:

# =============================================================================
# STEP 5. A5 테이블 생성 및 최종 컬럼 재정렬
# A3와 A4를 'RECAP_PK'를 기준으로 병합(왼쪽 조인)합니다.
# 병합 후, RECAP_YN과 ENGY_RECAP_YN 컬럼의 결측치는 0으로 대체합니다.
# 원하는 최종 컬럼 순서대로 재배치합니다.
# =============================================================================

A5 = pd.merge(A3, A4, on='RECAP_PK', how='left')

# 결측치(NaN)를 0으로 대체하고, 정수형으로 변환
A5['RECAP_YN'] = A5['RECAP_YN'].fillna(0).astype(int)
A5['ENGY_RECAP_YN'] = A5['ENGY_RECAP_YN'].fillna(0).astype(int)

# 최종 컬럼 순서 지정 (예시 순서: 단지 PK, 건축물 구분, 건축물 종류, 에너지공급기관코드, 
# 단지 총괄표제부 플래그, 관리건축물 PK, 그룹 행 개수, 에너지 플래그 등)
final_columns = ['RECAP_PK', '건축물_구분', '건축물_종류', '에너지공급기관코드',
                 'RECAP_YN', 'MGM_BLD_PK', 'N', 'ENGY_YN', 'ENGY_MGM_YN', 'ENGY_RECAP_YN']
A5 = A5[final_columns]
A5

,RECAP_PK,건축물_구분,건축물_종류,에너지공급기관코드,RECAP_YN,MGM_BLD_PK,N,ENGY_YN,ENGY_MGM_YN,ENGY_RECAP_YN
0,11110-100180009,2,3,1100100000,0,11110-100180009,5,1,1,0
1,11110-100180009,2,3,1200800000,0,11110-100180009,1,1,1,0
2,11110-100192417,1,2,1100100000,0,11110-100192417,8,1,1,0
3,11110-100196007,1,2,1100100000,0,11110-100196007,7,1,1,0
4,11110-100196007,1,2,1200800000,0,11110-100196007,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...
4491,52800-100182279,1,2,1203600000,0,52800-100182279,3,1,1,0
4492,52800-100182788,1,2,1100100000,0,52800-100182788,3,1,1,0
4493,52800-18617,1,2,1100100000,0,52800-18617,4,1,1,0
4494,52800-4852,1,2,1100100000,0,52800-4852,4,1,1,0


In [ ]:

# =============================================================================
# STEP 6. 최종 결과 A5를 CSV 파일로 저장
# =============================================================================
A5.to_csv("A5.csv", index=False)

# =============================================================================
# 코드 요약:
# - A1: 관리건축물(PK) 기준 그룹화 후 에너지 관련 플래그(ENGY_YN, ENGY_MGM_YN) 추가
# - A2: 단지(RECAP_PK) 기준 그룹화하여 총괄표제부(건축물_종류 == 1)만 추출, ENGY_RECAP_YN 추가
# - A3: A1에서 표제부(건축물_종류가 2 또는 3) 추출 후, A2와 병합하여 각 표제부의 단지에 총괄표제부 정보 추가
# - A4: 단지별 총괄표제부 존재 여부(RECAP_YN)를 생성 (중복 제거 후)
# - A5: A3와 A4 병합, 결측치 처리, 최종 컬럼 순서 재정렬 후 CSV로 저장
# =============================================================================


In [2]:
import pandas as pd
import os

# 결과 디렉토리 경로
result_dir = r"C:\Users\HyeonPoong Lee\REB_GR\REB_green-remodeling-project\02_matching_Econsumption_meter\result"

# 파일 경로 설정
matching_file = os.path.join(result_dir, "매칭정보.csv")
merged_file = os.path.join(result_dir, "merged_energy_data.xlsx")

# 파일 읽기
matching_df = pd.read_csv(matching_file, encoding='utf-8-sig')
merged_df = pd.read_excel(merged_file)

# MGM_BLD_PK의 고유값 추출
matching_pks = set(matching_df['MGM_BLD_PK'].unique())
merged_pks = set(merged_df['MGM_BLD_PK'].unique())

# 교집합 계산
common_pks = matching_pks.intersection(merged_pks)

# 결과 출력
print(f"매칭정보.csv의 고유 MGM_BLD_PK 수: {len(matching_pks):,}개")
print(f"merged_energy_data.xlsx의 고유 MGM_BLD_PK 수: {len(merged_pks):,}개")
print(f"두 파일에서 공통으로 존재하는 MGM_BLD_PK 수: {len(common_pks):,}개")

# 교집합 비율 계산
matching_ratio = len(common_pks) / len(matching_pks) * 100
merged_ratio = len(common_pks) / len(merged_pks) * 100

print(f"\n매칭정보.csv 기준 매칭률: {matching_ratio:.2f}%")
print(f"merged_energy_data.xlsx 기준 매칭률: {merged_ratio:.2f}%")

매칭정보.csv의 고유 MGM_BLD_PK 수: 8,652개
merged_energy_data.xlsx의 고유 MGM_BLD_PK 수: 3,038개
두 파일에서 공통으로 존재하는 MGM_BLD_PK 수: 1,657개

매칭정보.csv 기준 매칭률: 19.15%
merged_energy_data.xlsx 기준 매칭률: 54.54%


In [4]:
# import pandas as pd
# import os

# # 결과 디렉토리 경로
# result_dir = r"C:\Users\HyeonPoong Lee\REB_GR\REB_green-remodeling-project\02_matching_Econsumption_meter\result"

# # 파일 경로 설정
# matching_file = os.path.join(result_dir, "매칭정보.csv")
# merged_file = os.path.join(result_dir, "merged_energy_data.xlsx")

# # 파일 읽기
# print("파일 읽는 중...")
# matching_df = pd.read_csv(matching_file, encoding='utf-8-sig')
# merged_df = pd.read_excel(merged_file)


# 공통 MGM_BLD_PK만 필터링
common_pks = set(matching_df['MGM_BLD_PK']).intersection(set(merged_df['MGM_BLD_PK']))
merged_df = merged_df[merged_df['MGM_BLD_PK'].isin(common_pks)]
matching_df = matching_df[matching_df['MGM_BLD_PK'].isin(common_pks)]

print(f"처리할 건물 수: {len(common_pks):,}개")

# 결과를 저장할 리스트
result_rows = []

# 각 MGM_BLD_PK에 대해 처리
for pk in common_pks:
    # merged_energy_data에서 해당 PK의 행 가져오기
    merged_row = merged_df[merged_df['MGM_BLD_PK'] == pk].iloc[0]
    
    # 매칭정보에서 해당 PK의 모든 행 가져오기
    matching_rows = matching_df[matching_df['MGM_BLD_PK'] == pk]
    
    # 매칭정보가 없는 경우 처리
    if len(matching_rows) == 0:
        result_rows.append(merged_row.to_dict())
        continue
    
    # 각 매칭정보 행에 대해 merged 정보 추가
    for _, matching_row in matching_rows.iterrows():
        row_dict = merged_row.to_dict()
        row_dict.update(matching_row.to_dict())
        result_rows.append(row_dict)

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(result_rows)

# 결과 저장
output_file = os.path.join(result_dir, "merged_with_matching_info.xlsx")
result_df.to_excel(output_file, index=False)

print(f"\n처리 완료!")
print(f"원본 merged_energy_data 행 수: {len(merged_df):,}")
print(f"매칭 후 최종 행 수: {len(result_df):,}")
print(f"결과 파일 저장됨: {output_file}")

# 각 건물별 매칭정보 수 확인
matching_counts = result_df.groupby('MGM_BLD_PK').size()
print(f"\n건물별 매칭정보 수 통계:")
print(f"최소 매칭정보 수: {matching_counts.min():,}")
print(f"최대 매칭정보 수: {matching_counts.max():,}")
print(f"평균 매칭정보 수: {matching_counts.mean():.2f}")

처리할 건물 수: 1,657개

처리 완료!
원본 merged_energy_data 행 수: 1,657
매칭 후 최종 행 수: 25,866
결과 파일 저장됨: C:\Users\HyeonPoong Lee\REB_GR\REB_green-remodeling-project\02_matching_Econsumption_meter\result\merged_with_matching_info.xlsx

건물별 매칭정보 수 통계:
최소 매칭정보 수: 1
최대 매칭정보 수: 1,288
평균 매칭정보 수: 15.61
